In [5]:
import ipywidgets as widgets
import anywidget
import traitlets
import jupyter
from tweet_browser_test import tweet_browser as tb
import voila
from matplotlib import pyplot as plt
from IPython.display import display
import pandas as pd
import io


out = widgets.Output()

def startSession(file):
    if file['type'] == 'xls':
        df = pd.read_excel(io.BytesIO(file.content))
    else:
        df = pd.read_csv(io.BytesIO(file.content))
    db = tb.DataBaseSim(df)
    s = tb.Session(db)
    browser = Browser(s, out)
    

def selectColumns (row, colHeaders: list):
    result = []
    for j in colHeaders:
        result.append(row[s.headerDict[j]])
    return result

class SearchBar(anywidget.AnyWidget):
    _esm = "searchBar.js"
    _css = "searchBar.css"
    value = traitlets.List().tag(sync=True)
    

class Browser:
    def __init__(self, s, out):
        self.s = s
        self.out = out
        self.operators = [None] * 5
        self.colHeaders = list(s.headerDict.keys())
        self.createWidgets()
        self.resetDisplay()
        
    def resetDisplay(self, b = None):
        self.out.clear_output(True)
        with out:
            display(self.searchBar)
            display(self.searchButton)
            
    def searchKeyword(self, b):
        if(self.searchBar.value == ""):
            return
        self.s.searchKeyword(self.searchBar.value, True)
        self.getTweets(b)

    
    def getTweets(self, b):
        #self.resetDisplay()
        htmlText = "<style>table, th, td {border: 1px solid black;} th, td {padding: 10px;}</style>"
        htmlText += "<table> <tr> "
        for header in self.colHeaders:
        #for header in self.columns.value:
            htmlText += " <th> " + header + " </th> "
        dataSet = self.s.getCurrentSubset()
        for i in range(len(dataSet)):
            htmlText += " <tr> "
            for j in self.colHeaders:
            #for j in self.columns.value:
                #print(type(dataSet.iloc[i].loc[j]))
                htmlText += " <td>" + str(dataSet.iloc[i].loc[j]) + "</td> "
            htmlText += " </tr> "
        table = widgets.HTML(value=htmlText + " </table>")
        with out:
            print("Tweets displayed:", self.s.currentSet.size)
            display(table)
        
    def createWidgets(self):
        self.searchBar = SearchBar() 
        self.searchButton = widgets.Button(description='Search')
        self.searchButton.on_click(self.searchKeyword)
        

# File upload
fileUp = widgets.widgets.FileUpload(
    accept='.csv, .txt, .xls, .tsv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
    
def fileHandler(change):
    #print(change)
    #print(fileUp.value[0])
    #print(fileUp.value[0]['type'])
    startSession(fileUp.value[0])
        
with out:
    display(fileUp)
    #display(SearchBar())

fileUp.observe(fileHandler, names="value")
out
        


Output()